In [1]:
print("OK")

OK


In [2]:
import os
from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers
# from langchain.retrievers import BaseRetriever
from typing import List, Dict

In [3]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [4]:
extracted_data = load_pdf("data/")

In [5]:
extracted_data

[Document(page_content='Abdominal aorta ultrasound seeAbdominal\nultrasound\nAbdominal aortic aneurysm seeAortic\naneurysm\nAbdominal hernia seeHernia\nAbdominal thrust seeHeimlich maneuver\nAbdominal ultrasound\nDefinition\nUltrasound technology allows doctors to “see”\ninside a patient without resorting to surgery. A transmit-ter sends high frequency sound waves into the body,where they bounce off the different tissues and organs toproduce a distinctive pattern of echoes. A receiver“hears” the returning echo pattern and forwards it to acomputer, which translates the data into an image on atelevision screen. Because ultrasound can distinguishsubtle variations between soft, fluid-filled tissues, it isparticularly useful in providing diagnostic images of theabdomen. Ultrasound can also be used in treatment.\nPurpose\nThe potential medical applications of ultrasound\nwere first recognized in the 1940s as an outgrowth of thesonar technology developed to detect submarines duringWorld War I

In [6]:
#Text Chunks
def text_splits(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks=text_splits(extracted_data)
print("Length of my chunk",len(text_chunks))

Length of my chunk 50170


In [8]:
# text_chunks

In [9]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()
embeddings_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_result = embeddings_model.encode("Hello world", convert_to_tensor=False)
print("Length", len(query_result))

Length 384


In [13]:
# query_result

In [14]:
texts = [t.page_content for t in text_chunks]
embeddings = [embeddings_model.encode(text) for text in texts]

# Convert embeddings to numpy array
embeddings_array = np.array(embeddings).astype('float32')

# Create a FAISS index
d = embeddings_array.shape[1]  # dimension of embeddings
index = faiss.IndexFlatL2(d)  # L2 distance index
index.add(embeddings_array)  # add vectors to the index

print("Data successfully indexed!")

Data successfully indexed!


In [15]:
# Function to perform similarity search
def search(query, k=5):
    query_embedding = embeddings_model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, k)  # search
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

In [16]:
query = "can i take steroids"
results = search(query, k=3)
for result in results:
    print(f"Text: {result[0]}, Distance: {result[1]}")

Text: using steroids for lengthy periods may also have prob-lems with wound healing, may gain weight, and may suf-fer mental problems. In children, growth may be slowed.Besides being inhaled, steroids may be taken by mouthor injected, to rapidly control severe asthma.
LEUKOTRIENE MODIFIERS. Leukotriene modifiers, Distance: 0.7097356915473938
Text: the reported rate of use is 6 to 11% among high school-aged males, including an unexpected number of
nonathletes, and about 2.5% among high school-aged females.
Pathophysiology
Anabolic steroids have androgenic effects (eg, changes in hair or in libido, aggressiveness) and anabolic
effects (eg, increased protein utilization, increased muscle mass). Androgenic effects cannot be separated
from the anabolic, but some anabolic steroids have been synthesized to minimize the androgenic effects., Distance: 0.7661857008934021
Text: people recover rapidly, although effects may not fade for 1 to 2 h.
Anabolic Steroids
Anabolic steroids are often used t

In [17]:
query ="i am having a five year old son he is suffering from cold,cough,throat infection can u give him a tablets"
results = search(query, k=3)
# print(result[0])
for result in results:
    print(result[0])

symptom relief for children. And there are serious risks to consider . For
example, the sedating ef fects of antihistamines can be dangerous for kids
already having trouble breathing. For young children, an accidental
overdose of cough or cold medicine could be fatal.
The Food and Drug Administration (FDA) strongly encourages parents
to avoid cough and cold medicines for children younger than   years of age,
unless a health care professional has instructed you to use one.
ingredients that may alleviate coughing, but the amounts are too small
to do much good and may actually be harmful for children.
The college recommends against using nonprescription cough
syrups or cold medicines for anyone younger than age   . The Food
and Drug Administration strongly recommends against giving cough
and cold medicines to children younger than age  .
Under an agreement announced by manufacturers in     , several
brands of nonprescription cold and cough medications began carrying
In the hospital, your 

In [18]:
query ="how can i prevent from parkinson's and list me the medication"
results = search(query, k=3)
# print(result[0])
for result in results:
    print(result[0])

Treatment
• Carbidopa/levodopa (mainstay of treatment)
• Amantadine, monoamine oxidase type B (MAO-B) inhibitors, or anticholinergic drugs used first as
monotherapy or late with levodopa
• Dopamine agonists at any stage
• Catechol 
O
-methyltransferase (COMT) inhibitors sometimes used with levodopa
• Surgery if drugs are ineffective
• Exercise and adaptive measures
Many oral drugs are commonly used to relieve symptoms of Parkinson's disease (see
Table 183-3
believed to act by direct stimulation of the dopamine recep-tors in the brain. They may be used alone in early Parkison’sdisease, or as adjuncts to levodopa in advanced stages.
Recommended dosage
Dosages of anti-Parkinsonian medications must be
highly individualized. All doses must be carefully titrat-ed. Consult specific references.
Precautions
There are a large number of drugs and drug classes
used to treat Parkinson’s disease, and individual refer-ences should be consulted.
have years of active living after developing this diseas

In [19]:
def summarize_text(text, max_length=100):
    # A simple summarization method (this is a placeholder)
    # For better results, consider using an NLP library like Hugging Face's transformers for summarization
    sentences = text.split('. ')
    summarized = ' '.join(sentences[:2])  # Taking the first two sentences as an example
    return summarized[:max_length]  # Truncate to max_length
query = "What is acene."
results = search(query, k=3)

# Summarize and print the results
for result in results:
    text, distance = result[0], result[1]
    summarized_text = summarize_text(text)
    # print(f"Text: {summarized_text}, Distance: {distance}")
    print(summarized_text)
def summarize_text(text, max_length=100):
    # A simple summarization method
    sentences = text.split('. ')
    summarized = ' '.join(sentences[:2])  # Taking the first two sentences as an example
    return summarized[:max_length]  # Truncate to max_length

# def search(query, k=3):
#     # This function should perform the actual search and return results
#     # For this example, it's a placeholder and should be replaced with actual search logic
#     return [
#         ("GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies Allergic rhinitis is commonly triggered by exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen. The presence of an allergen causes the body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE. IgE molecules attach to mast cells, which contain histamine.HistaminePollen grains Lymphocyte FIRST EXPOSURE", 0.6048821210861206),
#         ("itchy, scratchy nose, eyes, and throat common in aller-gic rhinitis. The number of possible airborne allergens is enor-", 0.6296976208686829),
#         ("allergens are the following: • plant pollens • animal fur and dander • body parts from house mites (microscopic creatures found in all houses) • house dust• mold spores• cigarette smoke• solvents• cleaners Common food allergens include the following: • nuts, especially peanuts, walnuts, and brazil nuts • fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives The following types of drugs commonly cause aller- gic reactions: • penicillin or other antibiotics", 0.6369765996932983)
#     ]

# query = "What are Allergies."
# results = search(query, k=3)

# # Summarize and print the results
# for result in results:
#     text, distance = result[0], result[1]
#     summarized_text = summarize_text(text)
#     print(f"Text: {summarized_text}, Distance: {distance}")


animal, usually delivered through a bite or a sting.ACE is to help regulate arterial pressure by con
called ACE inhibitors) are medicines that block the con-version of the chemical angiotensin I to a s
normally have very high ACE levels Decreased levels maybe seen in the condition of excess fat in the


In [20]:
query = "What is achalasia"
results = search(query, k=3)
for result in results:
    print(f"Text: {result[0]}, Distance: {result[1]}")

Text: Check with a physician or pharmacist before combining
acetaminophen with any other prescription or nonpre-scription (over-the-counter) medicine.
Nancy Ross-Flanigan
Acetylsalicylic acid seeAspirin
Achalasia
Definition
Achalasia is a disorder of the esophagus that pre-
vents normal swallowing.
Description
Achalasia affects the esophagus, the tube that carries, Distance: 0.6180734634399414
Text: If you have signs or symptoms of achalasia, your doctor may order a
series of tests, possibly including a barium X-ray or an esophageal muscle, Distance: 0.6544289588928223
Text: includes barium swallow, endoscopy, and sometimes manometry. Treatments include dilation,
chemical denervation, and surgical myotomy.
Achalasia is thought to be caused by a loss of ganglion cells in the myenteric plexus of the esophagus,
resulting in denervation of esophageal muscle. Etiology of the denervation is unknown, although a viral
cause is suspected, and certain tumors may cause achalasia either by direct 

In [21]:
query = '''why disesase i have i am using these medication Tab.PCM 500mg fever, headache , body pains vachinappudu
 Montek LC 9pm for 3 days
 Strepcils(throat losengen) irritation vachinappuadalla 
 Limce 500mg 8 am 1pm 8 pm. Saparinchu oka 2 days'''
results = search(query, k=3)
for result in results:
    print(f"Text: {result[0]}")

Text: illness. Fever, usually 38.5 to 40° C, with rigors is accompanied by malaise, weakness, myalgia
(especially lumbar), retro-orbital headache, photophobia, anorexia, nausea, and light-headedness. Sore
throat and dysesthesia occur less often. After 5 days to 3 wk, patients may improve for 1 or 2 days. Many
relapse with recurrent fever, headache, rashes, swelling of metacarpophalangeal and proximal
interphalangeal joints, meningeal signs, orchitis, parotitis, or alopecia of the scalp.
Text: serious blood disorders. Anyone taking this medicineshould have regular blood counts and should check witha physician if any of the following symptoms occur:
• joint or muscle pain
• muscle weakness
• pain in the chest or abdomen
• tremors
• wheezing
• cough
• palpitations
• rash, sores, or pain in the mouth
• sore throat
• fever and chills
• loss of appetite
•diarrhea
• dark urine
• yellow skin or eyes
• unusual bleeding or bruising• dizziness
•hallucinations
• depression
Special conditions
Text:

In [22]:
query = '''syrups that can be used for fever.cough,cold,vomits for 2 years baby'''
results = search(query, k=3)
for result in results:
    print(result[0])

severe headache, or a persistent stomachache or vomiting.
If you think your child will be more comfortable if you lower the fever ,
you can use children’ s acetaminophen (T ylenol, others) or ibuprofen (Ad vil,
Motrin, others) to help reduce it. Extra fluids and minimal clothing also
may help. Don’ t use aspirin to reduce a fever in a child, unless specifically
prescribed by your child’ s care provider . Aspirin use in children has been
foods. Antibiotics may be prescribed if you have a high fever .
(eg, aspirin, naproxen) are also effective antipyretics. Salicylates should not be used to treat fever in
children with viral illnesses because use of salicylates has been associated with Reye's syndrome.
If temperature is ≥ 41° C, other cooling measures (eg, evaporative cooling with tepid water mist, cooling
blankets) should also be started.
Geriatrics Essentials
In the frail elderly, infection is less likely to cause fever, and even when elevated by infection, temperature


In [23]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [24]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [25]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [26]:
from langchain.chains.retrieval_qa.base import BaseRetriever
from langchain.schema import Document
from pydantic import BaseModel, Field
from typing import Callable, List
import numpy as np

class MyCustomRetriever(BaseRetriever):
    search_function: Callable[[str, int], List[str]] = Field(..., exclude=True)

    def __init__(self, search_function: Callable[[str, int], List[str]]):
        super().__init__()
        self.search_function = search_function

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # Call your search function to get search results
        search_results = self.search_function(query, k=2)
        # Process search results to match the expected format
        processed_data = [Document(page_content=result[0], metadata={"score": result[1]}) for result in search_results]
        return processed_data

In [27]:
# Define your custom search function
def search(query: str, k: int = 2):
    query_embedding = embeddings_model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# Create an instance of your custom retriever
custom_retriever = MyCustomRetriever(search_function=search)

# Create the RetrievalQA instance using your custom retriever
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [28]:
print("Hello")

Hello


In [29]:
while True:
    user_input = input("Input Prompt: ")
    result = qa.invoke({"query": user_input})
    print("Response: ", result["result"])

Response:  If your baby is having frequent, watery stools with green stringy bits and now has a severe diaper rash, it's important to consult with your pediatrician to rule out any underlying medical conditions that may be causing these symptoms. However, in the meantime, here are some things you can do to help soothe and treat the rash:
* Change your baby's diaper frequently and gently wash the skin every time the diaper is changed.
* Use plain water toile soap and warm water to clean water to avoiding water towater instead of water to clean water toothere water to clean water towaters instead of water to clean water toile for toilex water to water toilee water toile water to clean water toothapy water to clean water to clean water to witch water to water toile water to water to cool water toilet water to clean water toileanol water to clean water towater instead of water to wash2 water to avoid using water to water to clean water toile water toilevel water tow water toileatherapy to 